# Исследование объявлений о продаже квартир

В вашем распоряжении данные сервиса Яндекс.Недвижимость — архив объявлений о продаже квартир в Санкт-Петербурге и соседних населённых пунктов за несколько лет. Нужно научиться определять рыночную стоимость объектов недвижимости. Ваша задача — установить параметры. Это позволит построить автоматизированную систему: она отследит аномалии и мошенническую деятельность. 

По каждой квартире на продажу доступны два вида данных. Первые вписаны пользователем, вторые — получены автоматически на основе картографических данных. Например, расстояние до центра, аэропорта, ближайшего парка и водоёма. 

## Откройте файл с данными и изучите общую информацию. 

In [1]:
import pandas as pd
data = pd.read_csv('/datasets/real_estate_data.csv', sep='\t')
import matplotlib.pyplot as plt 
import seaborn as sns
data_02 = data.copy()

display(data.head())

FileNotFoundError: [Errno 2] No such file or directory: '/datasets/real_estate_data.csv'

In [ ]:
data.info()

In [ ]:
data.hist(figsize=(15, 20));

## Предобработка данных

In [ ]:
data.columns

In [ ]:
data = (
    data.rename(columns={'cityCenters_nearest' : 'city_centers_nearest',
                         'parks_around3000' : 'parks_around_3000',
                         'ponds_around3000' : 'ponds_around_3000'})
)

In [ ]:
data.columns

In [ ]:
data.isna().sum()

In [ ]:
# check
# пропущенные значения бары

def pass_value_barh(df):
    try:
        (
            (df.isna().mean()*100)
            .to_frame()
            .rename(columns = {0:'space'})
            .query('space > 0')
            .sort_values(by = 'space', ascending = True)
            .plot(kind= 'barh', figsize=(19,6), rot = -5, legend = False, fontsize = 16)
            .set_title('Пример' + "\n", fontsize = 22, color = 'SteelBlue')    
        );    
    except:
        print('пропусков не осталось :) ')

In [ ]:
pass_value_barh(data)

In [ ]:
data['living_area'] = data['living_area'].round(0)
data['living_area'] = data['living_area'].fillna(data.groupby('rooms')['living_area'].transform('median'))
data['living_area'].sort_values().unique()

In [ ]:
data['kitchen_area'] = data['kitchen_area'].round(0)
data['kitchen_area'] = data['kitchen_area'].fillna(data.groupby('rooms')['kitchen_area'].transform('median'))
data['kitchen_area'].sort_values().unique()
#x = data['total_area']*0.9 <= data['living_area'] + data['kitchen_area']
#print(x.head(10))

- В столбце площадь кухни и жилая площадь пропуски скорее всего из-за невнимательности людей и носят скорее всего полностью случайный характер. 

In [ ]:
data.isna().sum()

In [ ]:
# check
data[data['total_area']*0.9 <= data['living_area'] + data['kitchen_area']].head()

In [ ]:
data['is_apartment'] = data['is_apartment'].fillna(False)
data['is_apartment'].unique()

- Здесь скорее всего из за того что в местах где апартаментов нет просто оставляли пропуски, их можно заменить значением False. 

In [ ]:
data['balcony'] = data['balcony'].fillna(0)
data['balcony'].unique()

- Здесь отсутствующие балконы можно заменить на 0, скорее всего люди пропускали этот пункт если балконов у них нет.

In [ ]:
data['parks_around_3000'] = data['parks_around_3000'].fillna(0)
data['parks_around_3000'].unique()         

In [ ]:
data['ponds_around_3000'] = data['ponds_around_3000'].fillna(0)
data['ponds_around_3000'].unique()

- Пропуски в колонках число парков и водоемов в радиусе 3 км. можно объяснить их отсутствием и можно заменить на 0. 

In [ ]:
data['locality_name'] = data['locality_name'].astype(str)

def locality_name_02(name_02, locality_02):
    for x in name_02:
        data['locality_name'] = data['locality_name'].str.replace(name_02, locality_02)
x_1 = 'поселок'
name = 'посёлок'
locality_name_02(x_1, name)
x_4 = 'посёлок городского типа'
x_5 = 'городской посёлок'
locality_name_02(x_5, x_4)


In [ ]:
# check
data['locality_name'].sort_values().unique()

In [ ]:
data['is_apartment'] = data['is_apartment'].astype(bool)
data['first_day_exposition'] = pd.to_datetime(data['first_day_exposition'], format='%Y-%m-%dT%H:%M:%S')
data.dropna(subset=['floors_total'])
data['floors_total'] = data['floors_total'].fillna(0).astype(int)
data['floors_total'].value_counts()
print(data.groupby('floors_total')['floor'].value_counts())
#data.info()

- Заменяем в колонке апартаменты с булевыми значениями, дата публикации и этажность дома на соответствующий им тип данных для упрощения с ними работы. 

In [ ]:
data['ceiling_height'] = data['ceiling_height'].replace(25, 2.5) 
data['ceiling_height'] = data['ceiling_height'].replace(24, 2.4)
data['ceiling_height'] = data['ceiling_height'].replace(26, 2.6)
data['ceiling_height'] = data['ceiling_height'].replace(23, 2.3)
data['ceiling_height'] = data['ceiling_height'].replace(27, 2.7) 
data['ceiling_height'] = data['ceiling_height'].replace(28, 2.8)
data['ceiling_height'] = data['ceiling_height'].replace(20, 2.0)
data['ceiling_height'] = data['ceiling_height'].replace(32, 3.2)
data['ceiling_height'] = data['ceiling_height'].replace(14, 1.4)
data['ceiling_height'] = data['ceiling_height'].replace(10, 1.0)
data['ceiling_height'] = data['ceiling_height'].replace(100, 1.0)
data = data.query('ceiling_height != 22.6 and ceiling_height != 10.3 and ceiling_height != 27.5')
data['ceiling_height'] = data['ceiling_height'].round(0)
print(data['ceiling_height'].value_counts().sort_values())

- Здесь большое значение пропусков, наврятли эта толька человечиский фактор так как площить жилья и кухни таких пропусков имеет меньше возможна была техническая ошибка либа сбой, заменить их медианой от какой нибуть другой категории не имеет возможности. Есть выбивающиеся значения которое скорее всего являются опечатками. 

In [ ]:
# check

# Показатели о кол-ве объявлений в датасете, минимальных и максимальных значениях 
# в выбранных параметрах о продаже квартир
# сырые данные

(
    data[['rooms', 'total_area', 'ceiling_height', 'days_exposition', 'last_price', 'living_area',  'kitchen_area',
          'floor', 'floors_total']]
    .apply (['count', 'min', 'max'])   
    .style.format("{:,.2f}")
)

In [ ]:
# check
data.rooms.value_counts().to_frame()

In [ ]:
# check
data.total_area.hist(bins = 150, figsize = (15,3));

In [ ]:
# check
data.total_area.hist(bins = 150, figsize = (15,3), range = (180,500));

In [ ]:
#Собрал всю работу над редкими и выбивающимися значениями здесь
data = data.query('rooms < 7 and rooms !=0')
data = data.query('25 < total_area < 200')
data = data.query('last_price < 25000000')
data = data.query('ceiling_height != 1.0 and ceiling_height != 5.0 and ceiling_height != 6.0 and ceiling_height != 8.0')
data = data.query('kitchen_area != 1.0 and kitchen_area != 2.0 and kitchen_area != 3.0')
data = data.query('floors_total != 1 and floors_total != 0 and floors_total < 30')
data['days_exposition'] = data['days_exposition'].fillna(0)#Заменил все NaN на 0 чтобы сохранить данные и знать какие квартиры еще не проданы 
data = data.query('days_exposition < 900')
import numpy as np
data['kitchen_area'] = data['kitchen_area'].fillna(0)
data = data.query('kitchen_area < 30')
data.loc[data["kitchen_area"] == 0, "kitchen_area"] = np.nan
data.loc[data["days_exposition"] == 0, "days_exposition"] = np.nan
#data.isna().sum()
data.floors_total.value_counts().to_frame()
#data.info()

In [ ]:
# check

# Значения параметров объектов недвижимости на разных квантилях

(
    data[['rooms', 'total_area', 'ceiling_height', 'days_exposition', 'last_price', 'living_area',  
        'kitchen_area', 'floor',   'floors_total']]
    .quantile([0.01, .5, .90, .9976])  
    .style.format("{:,.2f}")
)

(
    data[['rooms', 'total_area', 'ceiling_height', 'days_exposition', 'last_price', 'living_area',  'kitchen_area',
          'floor', 'floors_total']]
    .apply (['count', 'min', 'max'])   
    .style.format("{:,.2f}")
)

In [ ]:
# check

# Показатели о кол-ве объявлений в датасете, минимальных и максимальных значениях 
# в выбранных параметрах о продаже квартир


(
    data[['rooms', 'total_area', 'ceiling_height', 'days_exposition', 'last_price', 'living_area',  'kitchen_area',
          'floor', 'floors_total']]
    .apply (['count', 'min', 'max'])   
    .style.format("{:,.2f}")
)

## Посчитайте и добавьте в таблицу новые столбцы

In [ ]:
data['price_one_square_meter'] = data['last_price'] / data['living_area']
data['price_one_square_meter'] = data['price_one_square_meter'].round(1)

In [ ]:
data['weekday'] = pd.DatetimeIndex(data['first_day_exposition']).weekday
data['month'] = pd.DatetimeIndex(data['first_day_exposition']).month
data["year"] = data["first_day_exposition"].dt.to_period("Y")

In [ ]:
def categorize_floor(row):
    floor_01 = row['floor']
    floor_02 = row['floors_total']
    
    try:
        if floor_01 == floor_02:
            return 'последний'
        if floor_01 == 1:
            return 'первый'
        if floor_01 != floor_02:
            return'другой'
    except:  
        return 'x'
data['categorize_floors'] = data.apply(categorize_floor, axis=1)
#print(data.loc[:, ['floors_total', 'categorize_floors','floor']].head(60))
# Я тут решил проблему проста избавившись от одноэтажных зданий, 
 #их всего 23 значения и не факт что они не попали сюда случайно, по идеи проблемы быть не должно.   

In [ ]:
data['city_centers_nearest_km'] = data['city_centers_nearest'] / 1000
data['city_centers_nearest_km'] = data['city_centers_nearest_km'].round(0)

## Проведите исследовательский анализ данных 

### Изучение параметров объектов:

In [ ]:
fig, ax = plt.subplots()
data['total_area'].hist(bins=175, range=(0, 200))
ax.set_title('Общая площадь')
ax.set_xlabel('Площадь, м²')
ax.set_ylabel('Квартиры')
plt.show()
#print(data['total_area'].describe()) 
#print(data['total_area'].median())
print(data['total_area'].corr(data['living_area']))
print(data['total_area'].corr(data['kitchen_area']))
print(data['living_area'].corr(data['kitchen_area']))

In [ ]:
# check
data.info()

In [ ]:
print(data['living_area'].describe())
print(data['living_area'].median())
print(data['living_area'].value_counts().sort_values(ascending=False).head(3))

In [ ]:
# check
data.shape

In [ ]:
fig, ax = plt.subplots()
data['living_area'].hist(bins=100, range=(0, 100))
ax.set_title('Жилая площадь')
ax.set_xlabel('Площадь, м²')
ax.set_ylabel('Квартиры')
plt.show()

In [ ]:
fig, ax = plt.subplots()
data['kitchen_area'].hist(bins=30, range=(0, 35))
ax.set_title('Площадь кухни')
ax.set_xlabel('Площадь, м²')
ax.set_ylabel('Квартиры')
plt.show()
print(data['kitchen_area'].median())
#data['kitchen_area'].describe()


In [ ]:
# check
data.shape

In [ ]:
data['last_price'] = data['last_price'].round(0)
fig, ax = plt.subplots()
data['last_price'].hist(bins=10)
ax.set_title('Цена на момент снятия с публикации')
ax.set_xlabel('Цена')
ax.set_ylabel('Квартиры')
plt.show()

#data['last_price'].describe()

In [ ]:
fig, ax = plt.subplots()
data['rooms'].hist(bins=8, range=(1, 8))
ax.set_title('Число комнат')
ax.set_xlabel('Комнаты')
ax.set_ylabel('Квартиры')
plt.show()

#data['rooms'].describe()

In [ ]:
fig, ax = plt.subplots()
data['ceiling_height'].hist(bins=4, range=(1, 5))
ax.set_title('Высота потолков')
ax.set_xlabel('Высота, м')
ax.set_ylabel('Квартиры')
plt.show()

#data['ceiling_height'].describe()
#print(data['ceiling_height'].value_counts())

In [ ]:
fig, ax = plt.subplots()
data['floor'].hist(bins=25, range=(0, 25))
ax.set_title('Этаж квартиры')
ax.set_xlabel('Этаж')
ax.set_ylabel('Квартиры')
plt.show()

#data['floor'].describe()
#print(data['floor'].value_counts().sort_values())

In [ ]:
fig, ax = plt.subplots()
data['floors_total'].hist(bins=30, range=(0, 30))
ax.set_title('Всего этажей в доме')
ax.set_xlabel('Этаж')
ax.set_ylabel('Квартиры')
plt.show()
data['floors_total'].describe()
print(data['floors_total'].median())
print(data['floor'].corr(data['floors_total']))

- С данными по этажности дома где продаётся квартира сильные искажения, на них лучше не полагаться.

In [ ]:
fig, ax = plt.subplots()
data['categorize_floors'].hist(bins=4, range=(0, 3))
ax.set_title('Тип этажа квартиры')
ax.set_xlabel('Этаж')
ax.set_ylabel('Квартиры')
plt.show()


In [ ]:
fig, ax = plt.subplots()
data['city_centers_nearest_km'].hist(bins=10, range=(1, 100))
ax.set_title('Расстояние до центра города')
ax.set_xlabel('Расстояние, км')
ax.set_ylabel('Квартиры')
plt.show()

#data['city_centers_nearest_km'].describe()

In [ ]:
fig, ax = plt.subplots()
data['airports_nearest'].hist(bins=10, range=(1, 70000))
ax.set_title('Расстояние до ближайшего аэропорта ')
ax.set_xlabel('Расстояние, м')
ax.set_ylabel('Квартиры')
plt.show()

#data['airports_nearest'].describe()

In [ ]:
fig, ax = plt.subplots()
data['parks_nearest'].hist(bins=30, range=(1, 1500))
ax.set_title('Расстояние до ближайшего парка')
ax.set_xlabel('Расстояние, м')
ax.set_ylabel('Квартиры')
plt.show()

#data['parks_nearest'].describe()

In [ ]:
fig, ax = plt.subplots()
data['ponds_nearest'].hist(bins=15, range=(1, 1200))
ax.set_title('Расстояние до ближайшего водоёма')
ax.set_xlabel('Расстояние, м')
ax.set_ylabel('Квартиры')
plt.show()


In [ ]:
fig, ax = plt.subplots()
data['month'].hist(bins=13, range=(0, 13))
ax.set_title('Месяц публикации объявления')
ax.set_xlabel('Месяц')
ax.set_ylabel('Квартиры')
plt.show()

In [ ]:
fig, ax = plt.subplots()
data['weekday'].hist(bins=7, range=(0, 7))
ax.set_title('День недели публикации объявления')
ax.set_xlabel('День')
ax.set_ylabel('Квартиры')
plt.show()


- Есть проблемы с площадями квартиры, у общей и у жилой площади есть большие просадки и шумы хотя выборка достаточно большая, а у площади кухни таких просадок нет, на гистограмме максимального этажа есть очень большие перепады. Остальные соответствуют нормальному распределению кроме месяца и дня недели, но они скорее зависят от выходных дней и сезона отпусков.

In [ ]:
# check

# Показатели о кол-ве объявлений в датасете, минимальных и максимальных значениях 
# в выбранных параметрах о продаже квартир
# сырые данные

(
    data[['rooms', 'total_area', 'ceiling_height', 'days_exposition', 'last_price', 'living_area',  'kitchen_area',
          'floor', 'floors_total']]
    .apply (['count', 'min', 'max'])   
    .style.format("{:,.2f}")
)

### Изучение сколько дней проходит с появления объявления о продаже квартиры:

In [ ]:
data_floor = data.query('floor < 10')
floor_exposition = data_floor.pivot_table(index='floor', values='days_exposition')
floor_days_exposition_01 = data_floor.pivot_table(index='floor', values='days_exposition', aggfunc='median')
floor_exposition.columns = ['days_exposition_mean']
floor_days_exposition_01.columns = ['days_exposition_median']
floor_exposition_02 = floor_exposition.join(floor_days_exposition_01)
floor_exposition_02['days_exposition_mean'] = floor_exposition_02['days_exposition_mean'].round(1)
floor_exposition_02.plot(kind='bar', y='days_exposition_median', figsize=(10, 5))
display(floor_exposition_02.reset_index().sort_values(by='days_exposition_median'))

In [ ]:
data_floor['days_exposition'].hist()
print(data_floor['days_exposition'].describe())
print(data_floor['days_exposition'].median())


 - Многие квартиры могут продаваться очень быстра, часта за 3-4 дня, примерна от 100 дней покупки становятся сильно реже и на 800 стремятся к 0. Наиболее быстро продаются 5, 6 и 7 этаж, первый этаж продается дольше всех. После 9 этажа выборка становится значительна меньше и по ним сложна делать выводы. Медиана 94 дня, средняя 163 из-за больших выбросов, чаще всего продажи занимают не более 100 дней. 

In [ ]:
# check
# Моды на реальных данных

try:
    df_check = pd.read_csv('real_estate_data.csv', sep='\t') 
except:
    df_check = pd.read_csv('/datasets/real_estate_data.csv', sep = '\t')
    
df_check['days_exposition'].value_counts().to_frame().head(20).plot(kind = 'barh', figsize = (15,6), rot = 0);

### Изучение факторов больше всего влияющих на общую стоимость объекта?

In [ ]:
last_price_01 = data.pivot_table(index='floor', values=['last_price', 'total_area', 'living_area', 'kitchen_area' ], aggfunc='median')
#data.info()
print(last_price_01.head(10))

In [ ]:
data.plot(kind='scatter', x='total_area', y='last_price', figsize=(10, 5)) 

In [ ]:
data.plot(kind='scatter', x='living_area', y='last_price', figsize=(10, 5)); 

In [ ]:
data.plot(kind='scatter', x='kitchen_area', y='last_price', figsize=(10, 5)) 

- Цена стоимости по площадям как видна на графиках имеет большой разброс, тут скорее всего есть искажения. Делать конкретные выводы по таким показателям нельзя. Возможна для привлечения внимания намерена завышают площадь квартиры. 

In [ ]:
last_price_room = data.pivot_table(index='rooms', values=['last_price'], aggfunc='median')
#data.plot(kind='scatter', x='rooms', y='last_price', figsize=(10, 5)) 
last_price_room.plot(kind='bar', y='last_price', figsize=(10, 5));

- Чем больше комнат тем больше цена, но из-за разности выборки сложна будет оценить этот разрыв на данных с большим количеством комнат, сравнивать квартиры можно только с одной до четырех комнат.

In [ ]:
last_price_01.plot(kind='bar', y='last_price', figsize=(10, 5))

In [ ]:
last_price_floors = data.pivot_table(index='categorize_floors', values=['last_price'], aggfunc='median')
last_price_floors.plot(kind='bar', y='last_price', figsize=(8, 4));
print(last_price_floors) 

- Первый этаж является самым не популярным, cложна делать выводы по последнему этажу тут есть сильные искажения. 

In [ ]:
last_price_weekday = data.pivot_table(index='weekday', values=['last_price'], aggfunc='median').reset_index()
last_price_month = data.pivot_table(index='month', values=['last_price'], aggfunc='median').reset_index()
last_price_year = data.pivot_table(index='year', values=['last_price'], aggfunc='median')
groupby_year = data.pivot_table(index='year', values=['last_price'], aggfunc='count')
groupby_year.columns = ['count']
last_price_year = last_price_year.join(groupby_year).reset_index()
last_price_weekday.plot(x='weekday', y='last_price', style='o-', ylim=(4400000, 4750000), grid=True, figsize=(10, 6))
display(last_price_weekday.sort_values(by='last_price', ascending=False))

- Ближе к выходном дням и на самих выходных люди предпочитают выкладывать предложения с более низкой ценой. Стоимость квартир повышается с понедельника на вторник и дальше снижается.

In [ ]:
#last_price_month.plot(kind='bar', y='last_price', figsize=(10, 7))
display(last_price_month.sort_values(by='last_price', ascending=False).head(10))

In [ ]:
last_price_month.plot(x='month', y='last_price', style='o-',  grid=True, figsize=(10, 6))

- К середине года цены сильно проседают, но постепенно возвращаются к своим показателям к концу. Наибольшую цену они имеют в апреле.

In [ ]:
display(last_price_year.sort_values(by='last_price',  ascending=False))

In [ ]:
last_price_year.plot(x='year', y='last_price', style='o-', xlim=(44, 50), ylim=(3400000, 8382500),  grid=True, figsize=(10, 5));

- В 2014 году цены были наиболее высокие, но наблюдений недостаточно нельзя делать окончательные выводы тут нужна больше выборки и дополнительные исследования, наиболее низкие цены были в 2016 году после чего снова начали расти. 

### Изучение цены одного квадратного метра и числа объявлений.

In [ ]:
locality_name_median = data.pivot_table(index='locality_name', values='price_one_square_meter', aggfunc='median')
locality_name_price = data.pivot_table(index='locality_name', values=['days_exposition'], aggfunc= 'count')
locality_name_square_meter = locality_name_median.join(locality_name_price)
locality_name_square_meter['price_one_square_meter'] = locality_name_square_meter['price_one_square_meter'].round(1)
locality_name_square_meter = locality_name_square_meter.query('days_exposition > 50')
display(locality_name_square_meter.sort_values(by='days_exposition', ascending=False).head(10))

In [ ]:
data['price_one_square_meter'].hist(bins=15, range=(0, 800000))

- Если брать выборку начиная с 50 то самая низкое стоимостью квадратного метра в Сланцах, а самая высокая в Кудрово. 

### Изучение зависимости стоимости объектов от расстояния до центра города.

In [ ]:
locality_name_centers_01 = (data[data['locality_name'] == 'Санкт-Петербург'])
locality_name_centers_sp = locality_name_centers_01.pivot_table(index='city_centers_nearest_km', columns='locality_name', values='price_one_square_meter')
locality_name_centers_sp =locality_name_centers_sp.reset_index()
locality_name_centers_sp['price_centers_nearest'] = locality_name_centers_sp['Санкт-Петербург'] / locality_name_centers_sp['city_centers_nearest_km']
locality_name_centers_sp.rename(columns={'Санкт-Петербург' : 'price_one_square_m'}, inplace = True)
locality_name_centers_sp['price_centers_nearest'] = locality_name_centers_sp['price_centers_nearest'].round(1)
data_count = data.groupby('city_centers_nearest_km')['city_centers_nearest_km'].count()
locality_name_centers_sp['price_one_square_m'] = locality_name_centers_sp['price_one_square_m'].round(1)
display(locality_name_centers_sp.sort_values(by='city_centers_nearest_km', ascending=False).head(10))
display(data_count.sort_values(ascending=False).head(10))

In [ ]:
locality_name_centers_sp.plot(kind='bar', x='city_centers_nearest_km', y='price_one_square_m', style='o', grid=True, figsize=(12, 6))
x = locality_name_centers_sp['price_one_square_m'].median() 
y = locality_name_centers_sp['city_centers_nearest_km'].median()
print(x / y)

- Средняя цена километра 13000 за один квадратный метр. Цена квартиры увеличивается по мере приближения к центру, но люди видимо предпочитают все таки покупать квартиры на небольшом удалении нежили прилегающие в плотную к центру. Двадцать седьмой километр сильно выбивается, возможна есть проблемы с самими данными.

## Общий вывод

Из проведённого исследования можно сделать выводы что:
 - Возможна данные по площади квартиры и максимальному этажу в доме часта искажаются. Нужна обращать внимания на слишком низкие цены и большие площади и то сколько этажей в доме.
 - На цену сильна влияют количество комнат и удаленность квартиры от центра. Также значение имеет этаж, не популярными являются первые этажи их цена наиболее низкая, сложна сказать являются данные по последним этажам достоверными.
 - К середине года цены падают, но постепенно возвращаются к своим показателям к концу. Стоимость квартир повышается с понедельника на вторник и дальше снижается.
 - Наиболее низкие цены были в 2016 году после чего снова начали расти, по 2014 году наблюдений недостаточно, нельзя делать окончательные выводы тут нужна больше выборки и дополнительные исследования.
 - Больше всего предложений в Санкт-Петербурге, самая низкое стоимостью квадратного метра в Сланцах, а самая высокая в Кудрово.